# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção: (caso o site esteja preparado para evitar esse tipo de utilizaçao do selenium)

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [14]:
import pandas as pd

produtos_df = pd.read_excel('arquivos/buscas.xlsx')
display(produtos_df)
produtos_df.info()

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch 11,3000,3500
1,rtx 3060,zota galax,4000,4500


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome            2 non-null      object
 1   Termos banidos  2 non-null      object
 2   Preço mínimo    2 non-null      int64 
 3   Preço máximo    2 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 192.0+ bytes


In [2]:
from selenium import webdriver # Comandar navegador controlado
from selenium.webdriver.chrome.service import Service # responsável por iniciar e parar o chromedriver
from webdriver_manager.chrome import ChromeDriverManager # Gerenciar arquvio ChromeDriver

# Instalar arquivo temporário ChromeDrive a partir da versão atual do Chrome
servico = Service(ChromeDriverManager().install())

# Abrir navegador com tela maximizada
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# Criar navegador
nav = webdriver.Chrome(service=servico, options=options)

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 35.6MB/s]


In [23]:
from selenium.webdriver.common.by import By # Selecionar elementos
from selenium.webdriver.common.keys import Keys # Utilizar comandos do teclado

# Criando Function Para Listar Valores Encontrados Google Shopping

def pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max):

    # Entrar google shopping
    nav.get('https://shopping.google.com/?nord=1')

    # Pesquisar produto
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(nome_produto)
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(Keys.ENTER)

    # Filtrar Valor min
    nav.find_element(By.NAME, 'lower').send_keys(str(preço_min))
    # Filtrar Valor max
    nav.find_element(By.NAME, 'upper').send_keys(str(preço_max))
    # Aplicar filtro de valor
    nav.find_element(By.NAME, 'upper').send_keys(Keys.ENTER)
        
    # Listando elementos com informações do produto pesquisado
    info_produto = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    # Listando termos banidos separadamente
    termos_banidos = nomes_banidos.lower()
    termos_banidos = termos_banidos.split(' ')

    # Listando termos do nome do produto separadamente
    termos_produto = nome_produto.lower()
    termos_produto = termos_produto.split(' ')

    # Criando lista com os nome, preço e link do produto
    resultado_pesquisa = []
    for elemento in info_produto:
        
        # Selecionando elemento com nome do produto
        nome_produto = elemento.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

        # Verificando termo banido
        produto_banido = False
        for termo in termos_banidos:
            if termo in nome_produto:
                produto_banido = True
        
        # Verificando se o nome do produto bate com o pesquisado
        produto_aceito = True
        for termo in termos_produto:
            if termo not in nome_produto:
                produto_aceito = False

        # Filtrando nome do produto
        if not produto_banido and produto_aceito:

            # Selecionando elemento com preço do produto
            preço = elemento.find_element(By.CLASS_NAME, 'a8Pemb').text

            # Selecionando elemento com link do produto
            elemento_link_filho = elemento.find_element(By.CLASS_NAME, 'aULzUe')
            elemento_link_pai = elemento_link_filho.find_element(By.XPATH, '..')
            link = elemento_link_pai.get_attribute('href')
            
            resultado_pesquisa.append((nome_produto, preço, link))

    # Ordenando lsta crescentemente a partir do indice 1 da tupla
    resultado_pesquisa.sort(key= lambda x: x[1], reverse=False)
    return resultado_pesquisa


In [24]:
import time

# Criando Function Para Listar Valores Encontrados Buscape

def pesquisa_buscape(nav, nome_produto, nomes_banidos, preço_min, preço_max):

    # Entrar buscapé
    nav.get('https://www.buscape.com.br/')

    # Pesquisar produto
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(nome_produto)
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)

    time.sleep(3)
  
    # Listando elementos com informações do produto pesquisado
    info_produto = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard__1D3ve')

    # Listando termos banidos separadamente
    termos_banidos = nomes_banidos.lower()
    termos_banidos = termos_banidos.split(' ')

    # Listando termos do nome do produto separadamente
    termos_produto = nome_produto.lower()
    termos_produto = termos_produto.split(' ')

    # Criando lista com os nome, preço e link do produto
    resultado_pesquisa = []
    for elemento in info_produto:
        
        # Selecionando elemento com nome do produto
        nome_produto = elemento.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text.lower()

        # Verificando termo banido
        produto_banido = False
        for termo in termos_banidos:
            if termo in nome_produto:
                produto_banido = True
        
        # Verificando se o nome do produto bate com o pesquisado
        produto_aceito = True
        for termo in termos_produto:
            if termo not in nome_produto:
                produto_aceito = False

        # Filtrando nome do produto
        if not produto_banido and produto_aceito:

            # Selecionando elemento com preço do produto
            preço = elemento.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
            preço_format = preço.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preço_format = float(preço_format)

            # Filtrando Valores
            if preço_min <= preço_format <= preço_max:

                # Selecionando elemento com link do produto
                link = elemento.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb').get_attribute('href')
            
                resultado_pesquisa.append((nome_produto, preço, link))

    # Ordenando lsta crescentemente a partir do indice 1 da tupla
    resultado_pesquisa.sort(key= lambda x: x[1], reverse=False)
    return resultado_pesquisa


In [25]:
valores_encontrados = pd.DataFrame()

for linha in produtos_df.index:
    nomes_banidos = produtos_df.loc[linha, 'Termos banidos']
    nome_produto = produtos_df.loc[linha, 'Nome']
    preço_max = produtos_df.loc[linha, 'Preço máximo']
    preço_min = produtos_df.loc[linha, 'Preço mínimo']

    gshop = pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max)
    if gshop:
        gshop_df = pd.DataFrame(gshop, columns=['PRODUTO', 'PREÇO', 'LINK'])
        valores_encontrados = pd.concat([valores_encontrados, gshop_df])
    else:
        gshop_df = None

    buscape = pesquisa_buscape(nav, nome_produto, nomes_banidos, preço_min, preço_max)
    if buscape:
        buscape_df = pd.DataFrame(buscape, columns=['PRODUTO', 'PREÇO', 'LINK'])
        valores_encontrados = pd.concat([valores_encontrados, buscape_df])
    else:
        buscape_df = None

display(valores_encontrados)

,PRODUTO,PREÇO,LINK
0,iphone 12 64gb branco | celular apple | usado,"R$ 3.015,00",https://www.google.com/url?url=https://www.enj...
1,iphone 12 64gb unlocked used,"R$ 3.025,94 + impostos",https://www.google.com/url?url=https://www.eba...
2,vitrine apple iphone 12 64gb original - verde,"R$ 3.099,00",https://www.google.com/url?url=https://produto...
3,apple iphone 12 64gb vitrine original garantia...,"R$ 3.116,52",https://www.google.com/url?url=https://produto...
4,"vitrine: iphone 12 apple 64gb roxo tela 6,1 câ...","R$ 3.134,05",https://www.google.com/url?url=https://www.car...
...,...,...,...
35,pc gamer fácil intel core i7 3a geração 16gb g...,"R$ 4.462,54",https://www.google.com/url?url=https://www.mag...
36,pc gamer amd ryzen 5 5600g 16gb (rtx 3060 12gb...,"R$ 4.479,90",https://www.google.com/url?url=https://www.sho...
37,msi geforce rtx 3060 ventus 2x oc 12gb gddr6 g...,"R$ 4.483,23 + impostos",https://www.google.com/url?url=https://www.eba...
38,"computador pichau gamer skoll, intel i5-10400,...","R$ 4.499,92",https://www.google.com/url?url=https://www.pic...


In [5]:
print(produtos_df['Preço mínimo'][0])

3000
